# Import modules

In [15]:
import requests
from bs4 import BeautifulSoup
import json
import datetime

In [16]:
year = datetime.date.today().year
month = datetime.date.today().month
day = datetime.date.today().day

# Set the url

In [17]:
def setURL(url, num_page=-1):
    if num_page == -1:
        response = requests.get(str(url))
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        response = requests.get(str(url) + str(num_page))
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup

# Find the project title, description and URL

In [18]:
num_page = 2
for page_index in range(num_page):
    soup = setURL('https://openscience.eu/projects?page=', page_index)

    title_body = soup.find_all('h2')
    des_body = soup.find_all('div', class_='node__content')
    for index in range(len(title_body)):
        print(title_body[index].text)
        print('https://openscience.eu' + des_body[index].find('a').get('href'))
        print(des_body[index].text)



GreenspaceHack


https://openscience.eu/greenspace-hack-an-iot-and-crowdsource-enabled-mobile-app-to-design-healthier-and-smarter-green-spaces

 


Emerging evidence suggests that greenspace, whether used for recreation or physical activity (i.e. ‘green exercise’), can improve mental health and wellbeing. Indeed, individuals who use greenspaces or engage in green exercise have less mental distress, less anxiety and depression and healthier cortisol levels.  GreenspaceHack aims to facilitate better design of greenspaces to promote health by providing robust evidence on which characteristics of greenspace citizens value most and which contribute to promoting healthier urban environments at individual and local levels.


ORION


https://openscience.eu/orion-life-science-biomedicine

 


ORION is a 4-year project (runs from May 2017 to April 2021) that has received funding from the European Union’s Horizon 2020 research and innovation programme. ORION means “Open Responsible research and 

In [19]:
def find_location(url):
    soup = setURL(str(url))
    text = str(soup.find('div', class_='clearfix text-formatted field field--name-field-art-infobox field--type-text-long field--label-hidden field__item'))
    # print(text)
    # print('-'*30)
    if text == 'None':
        return 'None'
    else:
        end_index = text.find('<br/>')
        # print(text[:end_index])
        start_index = text[:end_index].rfind('>')

        # print('start_index', start_index)
        # print('end_index', end_index)
        # print(text[start_index+1:end_index].strip())
        return str(text[start_index+1:end_index].strip())


# Saving the JSON file

In [21]:
# TODO: Speedup the process
# issue: using soup twice to get the location is too slow. can we get the location in the first soup?

json_data = []
for page_index in range(num_page):
    soup = setURL('https://openscience.eu/projects?page=', page_index)

    title_body = soup.find_all('h2')
    des_body = soup.find_all('div', class_='node__content')
    for index in range(len(title_body)):
        project_URL = 'https://openscience.eu' + str(des_body[index].find('a').get('href'))
        json_data.append({"title":title_body[index].text.strip(),'Project URL':project_URL, 'description': des_body[index].text.strip(), 'location': find_location(project_URL)})
print(json_data[:5])

[{'title': 'GreenspaceHack', 'Project URL': 'https://openscience.eu/greenspace-hack-an-iot-and-crowdsource-enabled-mobile-app-to-design-healthier-and-smarter-green-spaces', 'description': 'Emerging evidence suggests that greenspace, whether used for recreation or physical activity (i.e. ‘green exercise’), can improve mental health and wellbeing. Indeed, individuals who use greenspaces or engage in green exercise have less mental distress, less anxiety and depression and healthier cortisol levels.\xa0 GreenspaceHack aims to facilitate better design of greenspaces to promote health by providing robust evidence on which characteristics of greenspace citizens value most and which contribute to promoting healthier urban environments at individual and local levels.', 'location': 'UK'}, {'title': 'ORION', 'Project URL': 'https://openscience.eu/orion-life-science-biomedicine', 'description': 'ORION is a 4-year project (runs from May 2017 to April 2021) that has received funding from the Europe

In [22]:
json_data = json.dumps(json_data, indent=4)  # 'indent' for pretty formatting

# Specify the file path where you want to save the JSON data
file_path = "./../data/Openscience_eu/" + str(year) + str(month) + str(day) + "_data.json"

# Write the JSON string to the file
with open(file_path, 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)